In [1]:
import pandas as pd
import json

# Dictionary to order months
month_order = {
    'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4,
    'Mayo': 5, 'Junio': 6, 'Julio': 7, 'Agosto': 8,
    'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
}

dataset = "filtered_dataset_2024_v4_gastos_papel.csv"
df = pd.read_csv(dataset)

# Check coordinates in the first few rows
print(df[['Institución Padre', 'Coordenadas Geográficas']].head())

# Function to extract month and year from the 'Periodo' column
def extract_month_year(df):
    df['Mes'] = df['Periodo'].apply(lambda x: x.split('/')[0] if isinstance(x, str) else None)
    df['Año'] = df['Periodo'].apply(lambda x: x.split('/')[1] if isinstance(x, str) else None)
    return df

# Function to sort months chronologically
def sort_months(df):
    unique_months = df['Mes'].unique()
    sorted_months = sorted(unique_months, key=lambda month: month_order[month])
    return sorted_months

# Function to identify if an institution is a municipality
def is_municipality(institution_name):
    return 'MUNICIPALIDAD' in institution_name.upper()

# Function to get coordinates based on the institution name
def get_coordinates(institution, df):
    institution = institution.upper()
    coordinates = df[df['Institución Padre'].str.upper() == institution]['Coordenadas Geográficas'].values
    if len(coordinates) > 0:
        lat, lon = coordinates[0].split(",")
        return float(lat.strip()), float(lon.strip())
    else:
        return None, None

# Function to group and calculate rankings by month
def calculate_rankings(df, sorted_months):
    results_by_year = {}

    # Iterate over each unique month and calculate rankings
    for month in sorted_months:
        month_data = df[df['Mes'] == month]

        if month_data.empty:
            continue

        year = month_data['Año'].iloc[0]
        month_data.loc[:, 'Institución Padre'] = month_data['Institución Padre'].str.upper()

        municipality_data = month_data[month_data['Institución Padre'].apply(is_municipality)]
        other_data = month_data[~month_data['Institución Padre'].apply(is_municipality)]

        # Group and filter by expense >= 1000
        municipality_expenses = municipality_data.groupby('Institución Padre')['Gasto'].sum().reset_index()
        municipality_expenses = municipality_expenses[municipality_expenses['Gasto'] >= 1000]

        other_expenses = other_data.groupby('Institución Padre')['Gasto'].sum().reset_index()
        other_expenses = other_expenses[other_expenses['Gasto'] >= 1000]

        # Calculate rankings
        top_10_municipalities_asc = municipality_expenses.sort_values('Gasto').head(10)
        top_10_municipalities_desc = municipality_expenses.sort_values('Gasto', ascending=False).head(10)

        top_10_others_asc = other_expenses.sort_values('Gasto').head(10)
        top_10_others_desc = other_expenses.sort_values('Gasto', ascending=False).head(10)

        def add_coordinates(row):
            lat, lon = get_coordinates(row['Institución Padre'], df)
            row['Latitud'] = lat
            row['Longitud'] = lon
            return row

        # Add coordinates
        top_10_municipalities_asc = top_10_municipalities_asc.apply(add_coordinates, axis=1)
        top_10_municipalities_desc = top_10_municipalities_desc.apply(add_coordinates, axis=1)
        top_10_others_asc = top_10_others_asc.apply(add_coordinates, axis=1)
        top_10_others_desc = top_10_others_desc.apply(add_coordinates, axis=1)

        if year not in results_by_year:
            results_by_year[year] = {}

        results_by_year[year][month] = {
            'Top 10 Municipalidades Ascendente': top_10_municipalities_asc.to_dict(orient='records'),
            'Top 10 Municipalidades Descendente': top_10_municipalities_desc.to_dict(orient='records'),
            'Top 10 Otros Ascendente': top_10_others_asc.to_dict(orient='records'),
            'Top 10 Otros Descendente': top_10_others_desc.to_dict(orient='records')
        }

    return results_by_year

# Main function to get JSON results
def get_results_json(df):
    df = extract_month_year(df)
    sorted_months = sort_months(df)
    results_by_year = calculate_rankings(df, sorted_months)
    results_json = json.dumps(results_by_year, indent=4, ensure_ascii=False)
    return results_json

# Save the JSON result to a file
results_json = get_results_json(df)
output_file = 'dashboard_data_generation.json'

with open(output_file, 'w', encoding='utf-8') as f:
    f.write(results_json)

print(f"The JSON file has been saved to {output_file}")


                                   Institución Padre  \
0  CENTRO REFERENCIAL DE SALUD DR SALVADOR ALLEND...   
1             CORP ADMINISTRATIVA DEL PODER JUDICIAL   
2                          I MUNICIPALIDAD DE CALAMA   
3                 I MUNICIPALIDAD DE CALERA DE TANGO   
4                          I MUNICIPALIDAD DE CANETE   

                 Coordenadas Geográficas  
0        -33.44534655,-70.74119011102081  
1  -53.15002819490814,-70.91143822982174  
2         -22.4634382,-68.92573347389403  
3         -33.6306199,-70.76159311969573  
4                  -37.79972,-73.3991188  
The JSON file has been saved to dashboard_data_generation.json
